# José de Jesús Tapia López

# NOTA: Recordar ejecutar cada ejercicio en diferentes notebooks

## Ejercicio 1: Problema del Productor-Consumidor con 1 productor y 2 consumidores usando MUTEX

In [ ]:
from threading import Thread, Lock
from queue import Queue
from random import randint

class Mutex:
    def __init__(self):
        self.lock = Lock()
    
    def bloquear(self):
        self.lock.acquire()
    
    def desbloquear(self):
        self.lock.release()

class Productor(Thread):
    def __init__(self,buffer,mutex):
        super().__init__()
        self.buffer = buffer
        self.mutex = mutex
        self.esta_despierto = False
        
    def producir(self):
        item = randint(0,101)
        print(f'Productor: Acabo de producir {item}')
        return item
    
    def añadir_al_buffer(self,item):
        self.buffer.put(item)
        print(f'Productor: Agregué {item} al buffer')
    
    def run(self):
        while True:
            self.mutex.bloquear()
            if not self.buffer.full():
                print(f'Productor: Hay espacio para nuevos items')
                item = self.producir()
                self.añadir_al_buffer(item)
            self.mutex.desbloquear()


class Consumidor(Thread):
    def __init__(self,buffer,mutex):
        super().__init__()
        self.buffer = buffer
        self.mutex = mutex
        self.esta_despierto = False
    
    def quitar_del_buffer(self):
        item = self.buffer.get()
        print(f'Consumidor: Obtuve {item} del buffer')
        return item
    
    def consumir(self,item):
        print(f'Consumidor: Consumí {item}')
    
    def run(self):
        while True:
            self.mutex.bloquear()
            if not self.buffer.empty():
                print(f'Consumidor: Hay items por consumir')
                item = self.quitar_del_buffer()
                self.consumir(item)
            self.mutex.desbloquear()

mutex = Mutex()
buffer = Queue(5) # Tamaño del buffer

p = Productor(buffer,mutex)
c1 = Consumidor(buffer,mutex)
c2 = Consumidor(buffer,mutex)


p.start()
c1.start()
c2.start()

p.join()
c1.join()
c2.join()



# Ejercicio 2: Composición de funciones, donde cada función es un hilo:

In [ ]:
from threading import Thread, Lock
import time
import numpy as np
import matplotlib.pyplot as plt
from math import sin, cos


def g(z):
    x = z.pop()
    y = 3*x**2 + 2*x - 1
    z.append(y)


def f(z):
    x = z.pop()
    y = np.sin(x)+np.cos(x)
    z.append(y)

# Queremos que vaya en intervalos de 0.25
x = np.linspace(-100,100,801)
z = [x]
hilo1 = Thread(target=g,args=(z,))
hilo2 = Thread(target=f,args=(z,))

hilo1.start()
hilo1.join()
hilo2.start()
hilo2.join()

z

In [ ]:
# Hacemos la composición a mano para verificar que obtenemos lo mismo
def fg(z):
    y = np.sin(3*x**2 + 2*x - 1)+np.cos(3*x**2 + 2*x - 1)
    return z

w = fg(z)
w

In [ ]:
# Comprobamos
w == z

# Ejercicio 3

In [ ]:
from threading import Thread, RLock
import time

class MaquinaChicles(Thread):
    def __init__(self,chicles, lock):
        self.chicles = chicles
        self.lock = RLock()
        super().__init__()
    
    def run(self):
        t = time.time()
        self.lock.acquire() 
        self.chicles += 1
        print("Hemos producido un chicle")  
        time.sleep(1)
        print("Tiempo de producción del chicle: " +str(time.time()-t))
        self.lock.release()
        
class MaquinaPolvo(Thread):
    def __init__(self,polvo, lock):
        self.polvo = polvo
        self.lock = RLock()
        super().__init__()
    
    def run(self):
        t = time.time()
        self.lock.acquire() 
        self.polvo += 1
        print("Hemos producido una porción de polvo para un chicle")  
        time.sleep(1)
        print("Tiempo de producción del polvo: " +str(time.time()-t))
        self.lock.release()

class Proceso1(Thread):
    def __init__(self,chicles,polvo,bolichicles,lock):
        self.chicles = chicles
        self.polvo = polvo
        self.bolichicles = bolichicles
        self.lock = RLock()
        super().__init__()
            
    def run(self):
        t = time.time()
        if self.chicles > 1 and self.polvo > 1:  
            self.lock.acquire()
            self.chicles -= 1
            self.polvo -= 1
            self.bolichicles += 1
            print("El chicle se ha cubierto de polvo para formar un bolichicle")  
            time.sleep(1)
            print("Tiempo en que se creó el bolichicle: " + str(time.time()-t))
            self.lock.release()
           

class Proceso2(Thread):
    def __init__(self,bolichicles,paquete,lock):
        self.bolichicles = bolichicles
        self.paquete = paquete
        self.lock = RLock()
        super().__init__()
            
    def run(self):
        t = time.time()
        if self.bolichicles > 1 and self.paquete > 1:  
            self.lock.acquire()
            self.bolichicles -= 1
            self.paquete += 1
            print("El bolichicle se han envuelto en un paquete")  
            time.sleep(1)
            print("Tiempo en que se empaquetó el bolichicle: " +str(time.time()-t))
            self.lock.release()
   

class Proceso3(Thread):
    def __init__(self,paquete,bolsa,lock):
        self.paquete = paquete
        self.bolsa = bolsa
        self.lock = RLock()
        super().__init__()
            
    def run(self):
        t = time.time()
        if self.paquete > 9:  
            self.lock.acquire()
            self.bolsa += 1
            self.paquete -= 10
            print("Se han puesto en una bolsa  10 paquetes de bolichicles")   
            time.sleep(1)
            print("Tiempo en que se embolsaron los 10 paquetes de bolichicles: " +str(time.time()-t))
            self.lock.release()
     


chicles = 9
polvo = 9
bolichicles = 9
paquete = 10
bolsa = 2

lock = RLock()

hilo1 = MaquinaChicles(chicles,lock)
hilo2 = MaquinaPolvo(polvo,lock)

hilo3 = Proceso1(chicles, polvo, bolichicles, lock)
hilo4 = Proceso2(bolichicles, paquete, lock)
hilo5 = Proceso3(paquete, bolsa, lock)

hilo1.start()
hilo2.start()
hilo3.start()
hilo4.start()
hilo5.start()
